In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)



short_names = ['MIA', 'LAR', 'KAN', 'TAM', 'PHI', 'BAL', 'GNB', 'LVR', 'HOU', 'BUF', 'DET', 'IND', 
               'MIN', 'NOR', 'NYG', 'WAS', 'DAL', 'SEA', 'CLE', 'CIN', 'JAX', 'ARI', 'CAR', 'PIT', 
               'ATL', 'LAC', 'DEN', 'TEN', 'NWE', 'CHI','NYJ','SFO']

# Correctly matched full names
full_names = ['Miami Dolphins', 'Los Angeles Rams', 'Kansas City Chiefs', 'Tampa Bay Buccaneers', 
              'Philadelphia Eagles', 'Baltimore Ravens', 'Green Bay Packers', 'Las Vegas Raiders', 
              'Houston Texans', 'Buffalo Bills', 'Detroit Lions', 'Indianapolis Colts', 
              'Minnesota Vikings', 'New Orleans Saints', 'New York Giants', 'Washington Commanders', 
              'Dallas Cowboys', 'Seattle Seahawks', 'Cleveland Browns', 'Cincinnati Bengals', 
              'Jacksonville Jaguars', 'Arizona Cardinals', 'Carolina Panthers', 'Pittsburgh Steelers', 
              'Atlanta Falcons', 'Los Angeles Chargers', 'Denver Broncos', 'Tennessee Titans', 
              'New England Patriots', 'Chicago Bears','New York Jets','San Francisco 49ers']

AFC = ['MIA', 'KAN', 'BAL', 'LVR', 'HOU', 'BUF', 'IND', 'CLE', 'CIN', 
             'JAX', 'PIT', 'LAC', 'DEN', 'TEN', 'NWE','NYJ']
NFC = ['LAR', 'TAM', 'PHI', 'GNB', 'DET', 'MIN', 'NOR', 'NYG', 'WAS', 
             'DAL', 'SEA', 'ARI', 'CAR', 'ATL', 'CHI','SFO']

Divisions = {
    'AFC East': ['MIA', 'BUF', 'NWE', 'NYJ'],
    'AFC North': ['BAL', 'CIN', 'CLE', 'PIT'],
    'AFC South': ['HOU', 'IND', 'JAX', 'TEN'],
    'AFC West': ['KAN', 'LVR', 'LAC', 'DEN'],
    
    'NFC East': ['DAL', 'NYG', 'PHI', 'WAS'],
    'NFC North': ['GNB', 'CHI', 'DET', 'MIN'],
    'NFC South': ['TAM', 'NOR', 'ATL', 'CAR'],
    'NFC West': ['LAR', 'SEA', 'ARI', 'SFO']
}


# Create the DataFrame
df = pd.DataFrame({'Short Name': short_names, 'Full Name': full_names})

# Get the page content
url = 'https://www.pro-football-reference.com/years/2024/passing.htm'
response = requests.get(url)
page_content = response.content

# Parse the HTML content
soup = BeautifulSoup(page_content, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'passing'})

# Convert the table to a pandas DataFrame
passes = pd.read_html(str(table))[0]

#clean data to contain relevant values and remove unnecessary rows
passes = passes[(passes.GS > 0) & (passes.Pos == 'QB')].reset_index(drop=True)
pass_f = passes[passes['Player'] != 'League Average']

#rename QB columns so they are unique to position
pass_f.rename(columns = {'Yds':'Pass Yards','TD':'Pass TD','Att':'Pass Attempts','1D':'Passing 1D','Y/A':'Pass Yards/Att','Lng':'Pass (Lng)'},inplace=True)
pass_f.drop(columns = ['Succ%'],inplace=True)


pass_f['Sk%'] = pass_f['Sk%'] / 100
pass_f['Cmp%'] = pass_f['Cmp%'] / 100
pass_f['TD%'] = pass_f['TD%'] / 100
pass_f['Int%'] = pass_f['Int%'] / 100

display(pass_f.head())


,Rk,Player,Age,Team,Pos,G,GS,QBrec,Cmp,Pass Attempts,Cmp%,Pass Yards,Pass TD,TD%,Int,Int%,Passing 1D,Pass (Lng),Pass Yards/Att,AY/A,Y/C,Y/G,Rate,QBR,Sk,Yds.1,Sk%,NY/A,ANY/A,4QC,GWD,Awards
0,1.0,Tua Tagovailoa,26.0,MIA,QB,1.0,1.0,1-0-0,23.0,37.0,0.622,338.0,1.0,0.027,0.0,0.000,11.0,80.0,9.1,9.68,14.7,338.0,101.0,49.3,3.0,19.0,0.0750,7.98,8.48,1.0,1.0,NaN
1,2.0,Matthew Stafford,36.0,LAR,QB,1.0,1.0,0-1-0,34.0,49.0,0.694,317.0,1.0,0.020,1.0,0.020,15.0,63.0,6.5,5.96,9.3,317.0,85.2,62.9,2.0,13.0,0.0392,5.96,5.47,0.0,0.0,NaN
2,3.0,Patrick Mahomes,29.0,KAN,QB,1.0,1.0,1-0-0,20.0,28.0,0.714,291.0,1.0,0.036,1.0,0.036,15.0,35.0,10.4,9.50,14.6,291.0,101.9,69.6,2.0,10.0,0.0667,9.37,8.53,0.0,0.0,NaN
3,4.0,Baker Mayfield,29.0,TAM,QB,1.0,1.0,1-0-0,24.0,30.0,0.800,289.0,4.0,0.133,0.0,0.000,15.0,32.0,9.6,12.30,12.0,289.0,146.4,86.2,1.0,9.0,0.0323,9.03,11.61,0.0,0.0,NaN
4,5.0,Jalen Hurts,26.0,PHI,QB,1.0,1.0,1-0-0,20.0,34.0,0.588,278.0,2.0,0.059,2.0,0.059,14.0,67.0,8.2,6.71,13.9,278.0,80.3,38.8,2.0,12.0,0.0556,7.39,6.00,0.0,0.0,NaN


In [31]:
# Get the page content
url = 'https://www.pro-football-reference.com/years/2024/rushing.htm'
response = requests.get(url)
page_content = response.content

# Parse the HTML content
soup = BeautifulSoup(page_content, 'html.parser')

# Find and remove the <tr> tag with the class 'over header'
for tr in soup.find_all('tr', {'class': 'over_header'}):
    tr.decompose()  # Remove the <tr> element from the soup

# Find the table
table = soup.find('table', {'id': 'rushing'})

# Convert the cleaned table to a pandas DataFrame
rush = pd.read_html(str(table), header=0)[0]

#rename RB columns so they are unique to position
rush.rename(columns = {'Tm':'Team','Att':'Rush Att','Yds':'Rush Yards','TD':'Rush TD','1D':'Rushing 1D','Y/A':'Rush Yards/Att','Y/G':'Rush Yards/Game','Lng':'Rush (Lng)','Fmb':'RB Fumble'},inplace=True)

rush.drop(columns = ['Succ%'],inplace=True)

display(rush.head())

,Rk,Player,Team,Age,Pos,G,GS,Rush Att,Rush Yards,Rush TD,Rushing 1D,Rush (Lng),Rush Yards/Att,Rush Yards/Game,RB Fumble
0,1,Joe Mixon,HOU,28,RB,1,1,30,159,1,9,13,5.3,159.0,0
1,2,Jordan Mason,SFO,25,RB,1,1,28,147,1,8,24,5.3,147.0,0
2,3,Rhamondre Stevenson,NWE,26,RB,1,1,25,120,1,8,17,4.8,120.0,1
3,4,Saquon Barkley,PHI,27,RB,1,1,24,109,2,6,34,4.5,109.0,0
4,5,Najee Harris,PIT,26,RB,1,1,20,70,0,3,20,3.5,70.0,0


In [19]:
# Get the page content
url = 'https://www.pro-football-reference.com/years/2024/receiving.htm'
response = requests.get(url)
page_content = response.content

# Parse the HTML content
soup = BeautifulSoup(page_content, 'html.parser')

# Find and remove the <tr> tag with the class 'over header'
for tr in soup.find_all('tr', {'class': 'over_header'}):
    tr.decompose()  # Remove the <tr> element from the soup

# Find the table
table = soup.find('table', {'id': 'receiving'})
# Convert the cleaned table to a pandas DataFrame
rec = pd.read_html(str(table), header=0)[0]

#clean data to contain relevant values and remove unnecessary rows
rec = rec[rec['Player'] != 'Player']
rec = rec[rec.Rec != '0'].reset_index(drop=True)

#convert necessary columns to float to calculate YAC
rec['Y/R'] = pd.to_numeric(rec['Y/R'])
rec['Y/Tgt'] = pd.to_numeric(rec['Y/Tgt'])
rec['Yards after Catch'] = rec['Y/R'] - rec['Y/Tgt']

#rename WR columns so they are unique to WR
rec.rename(columns = {'Tm':'Team','Yds':'Receiving Yards','TD':'Receiving TD','1D':'Receiving 1D','Y/A':'Rush Yards/Att','Y/G':'Receiving Yards/Game','Lng':'Receiving (Lng)','Fmb':'WR Fumble'},inplace=True)

rec.drop(columns = ['Succ%','Ctch%'],inplace=True)


display(rec.head())


,Rk,Player,Team,Age,Pos,G,GS,Tgt,Rec,Receiving Yards,Y/R,Receiving TD,Receiving 1D,Receiving (Lng),Y/Tgt,R/G,Receiving Yards/Game,WR Fumble,Yards after Catch
0,1,Cooper Kupp,LAR,31,WR,1,1,21,14,110,7.9,1,6,21,5.2,14.0,110.0,0,2.7
1,2,Isaiah Likely,BAL,24,TE,1,1,12,9,111,12.3,1,5,49,9.3,9.0,111.0,0,3.0
2,3,Chris Godwin,TAM,28,WR,1,1,8,8,83,10.4,1,7,24,10.4,8.0,83.0,0,0.0
3,4,Devaughn Vele,DEN,27,WR,1,0,8,8,39,4.9,0,1,9,4.9,8.0,39.0,0,0.0
4,5,De'Von Achane,MIA,23,RB,1,0,7,7,76,10.9,0,2,39,10.9,7.0,76.0,0,0.0


In [30]:
#import defense data


# Get the page content
url = 'https://www.pro-football-reference.com/years/2024/defense.htm'
response = requests.get(url)
page_content = response.content

# Parse the HTML content
soup = BeautifulSoup(page_content, 'html.parser')

# Find and remove the <tr> tag with the class 'over header'
for tr in soup.find_all('tr', {'class': 'over_header'}):
    tr.decompose()  # Remove the <tr> element from the soup

# Find the table
table = soup.find('table', {'id': 'defense'})


# Convert the cleaned table to a pandas DataFrame
defense = pd.read_html(str(table), header=0)[0]

#remove unwanted rows

i = defense[defense.Player == 'Player'].index
defense = defense.drop(i)

defense.rename(columns = {'Tm':'Team'},inplace=True)

#replace NaN with 0
defense = defense.replace(np.nan,0)

#merge defense df with df in first cell to get division/conference data
defense = pd.merge(defense,df, left_on ='Team',right_on = 'Short Name',how = 'left')

defense['Conference'] = defense['Short Name'].apply(lambda x: 'AFC' if x in AFC else 'NFC')
    
for index, row in defense.iterrows():
    for division, teams in Divisions.items():
        if row['Team'] in teams:
            defense.loc[index, 'Division'] = division

defense.drop(columns = ['Short Name','Rk'],inplace=True)

display(defense.head())
defense.to_csv(r"C:\Users\patrick.wool_storabl\Downloads\NFLDEFENSEDATA.csv",index=False)

,Player,Team,Age,Pos,G,GS,Int,Yds,TD,Lng,PD,FF,Fmb,FR,Yds.1,TD.1,Sk,Comb,Solo,Ast,TFL,QBHits,Sfty,Full Name,Conference,Division
0,Zack Baun,PHI,28,LB,1,1,0,0,0,0,0,0,0,0,0,0,2.0,15,11,4,1,2,0,Philadelphia Eagles,NFC,NFC East
1,Alex Anzalone,DET,30,LB,1,1,0,0,0,0,0,0,0,0,0,0,0.0,13,10,3,3,0,0,Detroit Lions,NFC,NFC North
2,Terrel Bernard,BUF,25,LB,1,1,0,0,0,0,0,0,0,0,0,0,0.0,11,10,1,0,0,0,Buffalo Bills,AFC,AFC East
3,Julian Blackmon,IND,26,S,1,1,0,0,0,0,0,0,0,0,0,0,0.0,13,10,3,0,0,0,Indianapolis Colts,AFC,AFC South
4,T.J. Edwards,CHI,28,LB,1,1,0,0,0,0,0,0,0,1,0,0,0.0,15,10,5,2,0,0,Chicago Bears,NFC,NFC North


In [33]:
#Combine offensive data into one table

offense = pd.concat([pass_f,rush,rec],ignore_index = True)

#Turn data into percentages

offense['Cmp%'] = offense['Cmp%'] / 100
offense['TD%'] = offense['TD%'] / 100
offense['Int%'] = offense['Int%'] / 100
offense['Sk%'] = offense['Sk%'] / 100

#Split QB Record column into W/L/T columns

offense[['Win', 'Loss', 'Tie']] = offense['QBrec'].str.split('-', expand=True)

offense.drop(columns = ['Rk','Age'],inplace = True)

#Replace NAN with 0

offense = offense.replace(np.nan,0)

#merge offense df with df in first cell to get division/conference data

offense = pd.merge(offense,df, left_on ='Team',right_on = 'Short Name',how = 'left')
offense.drop(columns = ['Short Name','Awards'],inplace=True)

offense['Conference'] = offense['Team'].apply(lambda x: 'AFC' if x in AFC else 'NFC')
    
for index, row in offense.iterrows():
    for division, teams in Divisions.items():
        if row['Team'] in teams:
            offense.loc[index, 'Division'] = division

display(offense.head())
offense.to_csv(r"C:\Users\patrick.wool_storabl\Downloads\NFLPLAYERDATA.csv",index=False)

,Player,Team,Pos,G,GS,QBrec,Cmp,Pass Attempts,Cmp%,Pass Yards,Pass TD,TD%,Int,Int%,Passing 1D,Pass (Lng),Pass Yards/Att,AY/A,Y/C,Y/G,Rate,QBR,Sk,Yds.1,Sk%,NY/A,ANY/A,4QC,GWD,Rush Att,Rush Yards,Rush TD,Rushing 1D,Rush (Lng),Rush Yards/Att,Rush Yards/Game,RB Fumble,Tgt,Rec,Receiving Yards,Y/R,Receiving TD,Receiving 1D,Receiving (Lng),Y/Tgt,R/G,Receiving Yards/Game,WR Fumble,Yards after Catch,Win,Loss,Tie,Full Name,Conference,Division
0,Tua Tagovailoa,MIA,QB,1.0,1.0,1-0-0,23.0,37.0,0.00622,338.0,1.0,0.00027,0.0,0.00000,11.0,80.0,9.1,9.68,14.7,338.0,101.0,49.3,3.0,19.0,0.000750,7.98,8.48,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,1,0,0,Miami Dolphins,AFC,AFC East
1,Matthew Stafford,LAR,QB,1.0,1.0,0-1-0,34.0,49.0,0.00694,317.0,1.0,0.00020,1.0,0.00020,15.0,63.0,6.5,5.96,9.3,317.0,85.2,62.9,2.0,13.0,0.000392,5.96,5.47,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,1,0,Los Angeles Rams,NFC,NFC West
2,Patrick Mahomes,KAN,QB,1.0,1.0,1-0-0,20.0,28.0,0.00714,291.0,1.0,0.00036,1.0,0.00036,15.0,35.0,10.4,9.50,14.6,291.0,101.9,69.6,2.0,10.0,0.000667,9.37,8.53,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,1,0,0,Kansas City Chiefs,AFC,AFC West
3,Baker Mayfield,TAM,QB,1.0,1.0,1-0-0,24.0,30.0,0.00800,289.0,4.0,0.00133,0.0,0.00000,15.0,32.0,9.6,12.30,12.0,289.0,146.4,86.2,1.0,9.0,0.000323,9.03,11.61,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,1,0,0,Tampa Bay Buccaneers,NFC,NFC South
4,Jalen Hurts,PHI,QB,1.0,1.0,1-0-0,20.0,34.0,0.00588,278.0,2.0,0.00059,2.0,0.00059,14.0,67.0,8.2,6.71,13.9,278.0,80.3,38.8,2.0,12.0,0.000556,7.39,6.00,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,1,0,0,Philadelphia Eagles,NFC,NFC East


In [22]:
#get standings/result data

# Get the page content
url = 'https://www.pro-football-reference.com/years/2024/games.htm'
response = requests.get(url)
page_content = response.content

# Parse the HTML content
soup = BeautifulSoup(page_content, 'html.parser')

# Find and remove the <tr> tag with the class 'over header'
for tr in soup.find_all('tr', {'class': 'over_header'}):
    tr.decompose()  # Remove the <tr> element from the soup

# Find the table
table = soup.find('table', {'id': 'games'})

# Convert the cleaned table to a pandas DataFrame
standings = pd.read_html(str(table), header=0)[0]
standings = standings.drop(columns = {"Unnamed: 7",'Unnamed: 5'})

i = standings[standings.Date == 'Date'].index
standings = standings.drop(i)

#Add in month column
standings['Date'] = pd.to_datetime(standings['Date'])
standings['Month'] = standings.Date.dt.month
standings['Day'] = standings['Date'].dt.strftime('%A')
standings['Time'] = pd.to_datetime(standings['Time'], format='%I:%M%p').dt.strftime('%I:%M%p')

#update week dtype to prep for merge

standings['Week'] = standings.Week.astype('int64')

sunday_afternoon = ['01:00PM', '03:00PM','04:25PM','04:30PM','04:15PM','04:05PM']
sunday_night = ['06:30PM','08:20PM','08:15PM','08:00PM']

# Create a new column 'game_name' and initialize it with 'Uncategorized'
standings['game_name'] = 'Uncategorized'

# Apply conditions for 'Thursday'
standings.loc[standings['Day'] == 'Thursday', 'game_name'] = 'Thursday Night'

# Apply conditions for 'Friday'
standings.loc[standings['Day'] == 'Friday', 'game_name'] = 'Friday Night'

# Apply conditions for 'Monday'
standings.loc[standings['Day'] == 'Monday', 'game_name'] = 'Monday Night'

# Apply conditions for 'Saturday'
standings.loc[standings['Day'] == 'Saturday', 'game_name'] = 'Saturday Night'

# Apply conditions for 'Sunday Afternoon'
standings.loc[(standings['Day'] == 'Sunday') & (standings['Time'].isin(sunday_afternoon)), 'game_name'] = 'Sunday Afternoon'

# Apply conditions for 'Sunday Night'
standings.loc[(standings['Day'] == 'Sunday') & (standings['Time'].isin(sunday_night)), 'game_name'] = 'Sunday Night'
#apply conditions for 'International'
standings.loc[((standings['Day'] == 'Sunday') |(standings['Day'] == 'Friday'))& ((standings['Time'] == '09:30AM')|(standings['Time'] == '08:15PM')), 'game_name'] = 'International'


#replace NaN with 0

standings = standings.replace(np.nan,0)
# Output the cleaned data

display(standings.head())


,Week,Day,Date,Time,Winner/tie,Loser/tie,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Month,game_name
0,1,Thursday,2024-09-05,08:20PM,Kansas City Chiefs,Baltimore Ravens,27,20,353,1,452,1,9,Thursday Night
1,1,Friday,2024-09-06,08:15PM,Philadelphia Eagles,Green Bay Packers,34,29,410,3,414,1,9,International
2,1,Sunday,2024-09-08,01:00PM,Pittsburgh Steelers,Atlanta Falcons,18,10,270,0,226,3,9,Sunday Afternoon
3,1,Sunday,2024-09-08,01:00PM,Buffalo Bills,Arizona Cardinals,34,28,352,1,270,1,9,Sunday Afternoon
4,1,Sunday,2024-09-08,01:00PM,New Orleans Saints,Carolina Panthers,47,10,379,1,193,3,9,Sunday Afternoon


In [23]:
import nfl_data_py as nfl
schedule = nfl.import_schedules([2024]).reset_index(drop=True).fillna(0)

#create month column

schedule['gameday'] = pd.to_datetime(schedule['gameday'])
schedule['Month'] = schedule.gameday.dt.month

#clean time formatting

schedule['gametime'] = pd.to_datetime(schedule['gametime'], format='%H:%M')
schedule['gametime'] = schedule['gametime'].dt.strftime('%I:%M %p').str.replace(' ','')

# create column for regular season and postseason

game = ['DIV','CON','SB']
schedule['Regular_Post'] = schedule.game_type.apply(lambda x:'POST' if x in game else 'REG')

#create list to use for logic in gametime column

sunday_afternoon = ['01:00PM', '03:00PM','04:25PM','04:30PM','04:15PM','04:05PM']
sunday_night = ['06:30PM','08:20PM','08:15PM','08:00PM']

# Create a new column 'game_name' and initialize it with 'Uncategorized'
schedule['game_name'] = 'Uncategorized'

# Apply conditions for 'Thursday'
schedule.loc[schedule['weekday'] == 'Thursday', 'game_name'] = 'Thursday Night'

# Apply conditions for 'Friday'
schedule.loc[schedule['weekday'] == 'Friday', 'game_name'] = 'Friday Night'

# Apply conditions for 'Monday'
schedule.loc[schedule['weekday'] == 'Monday', 'game_name'] = 'Monday Night'

# Apply conditions for 'Saturday'
schedule.loc[schedule['weekday'] == 'Saturday', 'game_name'] = 'Saturday Night'

# Apply conditions for 'Sunday Afternoon'
schedule.loc[(schedule['weekday'] == 'Sunday') & (schedule['gametime'].isin(sunday_afternoon)), 'game_name'] = 'Sunday Afternoon'

# Apply conditions for 'Sunday Night'
schedule.loc[(schedule['weekday'] == 'Sunday') & (schedule['gametime'].isin(sunday_night)), 'game_name'] = 'Sunday Night'
#apply conditions for 'International'
schedule.loc[((schedule['weekday'] == 'Sunday') |(schedule['weekday'] == 'Friday'))& ((schedule['gametime'] == '09:30AM')|(schedule['gametime'] == '08:15PM')), 'game_name'] = 'International'


# Original and new values
original_values = ['BAL', 'GB', 'PIT', 'ARI', 'TEN', 'NE', 'HOU', 'JAX', 'CAR', 'MIN', 'LV', 'DEN', 
                   'DAL', 'WAS', 'LA', 'NYJ', 'BUF', 'LAC', 'NO', 'TB', 'IND', 'CLE', 'SF', 'SEA', 
                   'NYG', 'CIN', 'CHI', 'ATL', 'PHI', 'MIA', 'DET', 'KC']

new_values = ['BAL', 'GNB', 'PIT', 'ARI', 'TEN', 'NWE', 'HOU', 'JAX', 'CAR', 'MIN', 'LVR', 'DEN', 
              'DAL', 'WAS', 'LAR', 'NYJ', 'BUF', 'LAC', 'NOR', 'TAM', 'IND', 'CLE', 'SFO', 'SEA', 
              'NYG', 'CIN', 'CHI', 'ATL', 'PHI', 'MIA', 'DET', 'KAN']

# Create a mapping dictionary
rename_mapping = dict(zip(original_values, new_values))


# Replace the original values with the new values
schedule['away_team'] = schedule['away_team'].replace(rename_mapping)
schedule['home_team'] = schedule['home_team'].replace(rename_mapping)

#merge schedule with df in first cell to prepare merge with standings df

schedule = pd.merge(schedule,df, left_on ='away_team',right_on = 'Short Name',how = 'left')
schedule = pd.merge(schedule,df, left_on ='home_team',right_on = 'Short Name',how = 'left')

#create column for winning and losing team

for i in range(len(schedule)):
    if schedule.loc[i,'away_score'] > schedule.loc[i,'home_score']:
        schedule.loc[i,'Winning_Team'] = schedule.loc[i,'Full Name_x']
    else:
        schedule.loc[i,'Winning_Team'] = schedule.loc[i,'Full Name_y']

#drop columns

schedule.drop(columns = ['old_game_id','gsis','nfl_detail_id','pfr','pff','espn','ftn','away_qb_id','home_qb_id','stadium_id'],inplace=True)

display(schedule.head())

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,temp,wind,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium,Month,Regular_Post,game_name,Short Name_x,Full Name_x,Short Name_y,Full Name_y,Winning_Team
0,2024_01_BAL_KC,2024,REG,1,2024-09-05,Thursday,08:20PM,BAL,20.0,KAN,27.0,Home,7.0,47.0,0.0,7,7,124.0,-148.0,3.0,-118.0,-102.0,46.0,-110.0,-110.0,0,outdoors,grass,67.0,8.0,Lamar Jackson,Patrick Mahomes,John Harbaugh,Andy Reid,Shawn Hochuli,GEHA Field at Arrowhead Stadium,9,REG,Thursday Night,BAL,Baltimore Ravens,KAN,Kansas City Chiefs,Kansas City Chiefs
1,2024_01_GB_PHI,2024,REG,1,2024-09-06,Friday,08:15PM,GNB,29.0,PHI,34.0,Neutral,5.0,63.0,0.0,7,7,110.0,-130.0,2.0,-110.0,-110.0,49.5,-112.0,-108.0,0,outdoors,0,0.0,0.0,Jordan Love,Jalen Hurts,Matt LaFleur,Nick Sirianni,Ron Torbert,Arena Corinthians,9,REG,International,GNB,Green Bay Packers,PHI,Philadelphia Eagles,Philadelphia Eagles
2,2024_01_PIT_ATL,2024,REG,1,2024-09-08,Sunday,01:00PM,PIT,18.0,ATL,10.0,Home,-8.0,28.0,0.0,7,7,160.0,-192.0,4.0,-110.0,-110.0,43.0,-115.0,-105.0,0,closed,fieldturf,0.0,0.0,Justin Fields,Kirk Cousins,Mike Tomlin,Raheem Morris,Brad Rogers,Mercedes-Benz Stadium,9,REG,Sunday Afternoon,PIT,Pittsburgh Steelers,ATL,Atlanta Falcons,Pittsburgh Steelers
3,2024_01_ARI_BUF,2024,REG,1,2024-09-08,Sunday,01:00PM,ARI,28.0,BUF,34.0,Home,6.0,62.0,0.0,7,7,250.0,-310.0,6.5,-105.0,-115.0,46.0,-112.0,-108.0,0,outdoors,a_turf,61.0,20.0,Kyler Murray,Josh Allen,Jonathan Gannon,Sean McDermott,Tra Blake,New Era Field,9,REG,Sunday Afternoon,ARI,Arizona Cardinals,BUF,Buffalo Bills,Buffalo Bills
4,2024_01_TEN_CHI,2024,REG,1,2024-09-08,Sunday,01:00PM,TEN,17.0,CHI,24.0,Home,7.0,41.0,0.0,7,7,164.0,-198.0,4.0,-108.0,-112.0,43.0,-110.0,-110.0,0,outdoors,grass,67.0,8.0,Will Levis,Caleb Williams,Brian Callahan,Matt Eberflus,Shawn Smith,Soldier Field,9,REG,Sunday Afternoon,TEN,Tennessee Titans,CHI,Chicago Bears,Chicago Bears


In [26]:
#Merge schedule/results tables

Schedule = pd.merge(schedule,standings,left_on = ['gameday','gametime','Winning_Team'],right_on = ['Date','Time','Winner/tie'],how = 'left')

display(Schedule.head())

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,temp,wind,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium,Month_x,Regular_Post,game_name_x,Short Name_x,Full Name_x,Short Name_y,Full Name_y,Winning_Team,Week,Day,Date,Time,Winner/tie,Loser/tie,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Month_y,game_name_y
0,2024_01_BAL_KC,2024,REG,1,2024-09-05,Thursday,08:20PM,BAL,20.0,KAN,27.0,Home,7.0,47.0,0.0,7,7,124.0,-148.0,3.0,-118.0,-102.0,46.0,-110.0,-110.0,0,outdoors,grass,67.0,8.0,Lamar Jackson,Patrick Mahomes,John Harbaugh,Andy Reid,Shawn Hochuli,GEHA Field at Arrowhead Stadium,9,REG,Thursday Night,BAL,Baltimore Ravens,KAN,Kansas City Chiefs,Kansas City Chiefs,1.0,Thursday,2024-09-05,08:20PM,Kansas City Chiefs,Baltimore Ravens,27,20,353,1,452,1,9.0,Thursday Night
1,2024_01_GB_PHI,2024,REG,1,2024-09-06,Friday,08:15PM,GNB,29.0,PHI,34.0,Neutral,5.0,63.0,0.0,7,7,110.0,-130.0,2.0,-110.0,-110.0,49.5,-112.0,-108.0,0,outdoors,0,0.0,0.0,Jordan Love,Jalen Hurts,Matt LaFleur,Nick Sirianni,Ron Torbert,Arena Corinthians,9,REG,International,GNB,Green Bay Packers,PHI,Philadelphia Eagles,Philadelphia Eagles,1.0,Friday,2024-09-06,08:15PM,Philadelphia Eagles,Green Bay Packers,34,29,410,3,414,1,9.0,International
2,2024_01_PIT_ATL,2024,REG,1,2024-09-08,Sunday,01:00PM,PIT,18.0,ATL,10.0,Home,-8.0,28.0,0.0,7,7,160.0,-192.0,4.0,-110.0,-110.0,43.0,-115.0,-105.0,0,closed,fieldturf,0.0,0.0,Justin Fields,Kirk Cousins,Mike Tomlin,Raheem Morris,Brad Rogers,Mercedes-Benz Stadium,9,REG,Sunday Afternoon,PIT,Pittsburgh Steelers,ATL,Atlanta Falcons,Pittsburgh Steelers,1.0,Sunday,2024-09-08,01:00PM,Pittsburgh Steelers,Atlanta Falcons,18,10,270,0,226,3,9.0,Sunday Afternoon
3,2024_01_ARI_BUF,2024,REG,1,2024-09-08,Sunday,01:00PM,ARI,28.0,BUF,34.0,Home,6.0,62.0,0.0,7,7,250.0,-310.0,6.5,-105.0,-115.0,46.0,-112.0,-108.0,0,outdoors,a_turf,61.0,20.0,Kyler Murray,Josh Allen,Jonathan Gannon,Sean McDermott,Tra Blake,New Era Field,9,REG,Sunday Afternoon,ARI,Arizona Cardinals,BUF,Buffalo Bills,Buffalo Bills,1.0,Sunday,2024-09-08,01:00PM,Buffalo Bills,Arizona Cardinals,34,28,352,1,270,1,9.0,Sunday Afternoon
4,2024_01_TEN_CHI,2024,REG,1,2024-09-08,Sunday,01:00PM,TEN,17.0,CHI,24.0,Home,7.0,41.0,0.0,7,7,164.0,-198.0,4.0,-108.0,-112.0,43.0,-110.0,-110.0,0,outdoors,grass,67.0,8.0,Will Levis,Caleb Williams,Brian Callahan,Matt Eberflus,Shawn Smith,Soldier Field,9,REG,Sunday Afternoon,TEN,Tennessee Titans,CHI,Chicago Bears,Chicago Bears,1.0,Sunday,2024-09-08,01:00PM,Chicago Bears,Tennessee Titans,24,17,148,1,244,3,9.0,Sunday Afternoon


In [34]:


Schedule.rename(columns = {'Winner/tie':'Winning Team','Loser/tie':'Losing Team','Month_x':'Month','game_name_y':'Game Name','Full Name_x':'Full Name Away','Full Name_y':'Full Name Home'},inplace=True)
#Schedule.drop(columns = ['game_name_x','Short Name_x','Short Name_y','Winning_Team','Week','Day','Date','Time','Month_y'],inplace=True)

#replace misssing data with 0
Schedule = Schedule.replace(np.nan,0)

#Remove games not yet played
Schedule = Schedule[Schedule['Game Name'] != 0].reset_index(drop=True)

#update surface column

for i in range(len(Schedule)):
    if Schedule.loc[i,'roof'] == 'outdoors':
        Schedule.loc[i,'surface'] = 'grass'
    else:
        Schedule.loc[i,'surface'] = 'artificial'


display(Schedule)
Schedule.to_csv(r"C:\Users\patrick.wool_storabl\Downloads\NFLGAMEDATA.csv",index=False)

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,temp,wind,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium,Month,Regular_Post,Full Name Away,Full Name Home,Winning Team,Losing Team,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Game Name
0,2024_01_BAL_KC,2024,REG,1,2024-09-05,Thursday,08:20PM,BAL,20.0,KAN,27.0,Home,7.0,47.0,0.0,7,7,124.0,-148.0,3.0,-118.0,-102.0,46.0,-110.0,-110.0,0,outdoors,grass,67.0,8.0,Lamar Jackson,Patrick Mahomes,John Harbaugh,Andy Reid,Shawn Hochuli,GEHA Field at Arrowhead Stadium,9,REG,Baltimore Ravens,Kansas City Chiefs,Kansas City Chiefs,Baltimore Ravens,27,20,353,1,452,1,Thursday Night
1,2024_01_GB_PHI,2024,REG,1,2024-09-06,Friday,08:15PM,GNB,29.0,PHI,34.0,Neutral,5.0,63.0,0.0,7,7,110.0,-130.0,2.0,-110.0,-110.0,49.5,-112.0,-108.0,0,outdoors,grass,0.0,0.0,Jordan Love,Jalen Hurts,Matt LaFleur,Nick Sirianni,Ron Torbert,Arena Corinthians,9,REG,Green Bay Packers,Philadelphia Eagles,Philadelphia Eagles,Green Bay Packers,34,29,410,3,414,1,International
2,2024_01_PIT_ATL,2024,REG,1,2024-09-08,Sunday,01:00PM,PIT,18.0,ATL,10.0,Home,-8.0,28.0,0.0,7,7,160.0,-192.0,4.0,-110.0,-110.0,43.0,-115.0,-105.0,0,closed,artificial,0.0,0.0,Justin Fields,Kirk Cousins,Mike Tomlin,Raheem Morris,Brad Rogers,Mercedes-Benz Stadium,9,REG,Pittsburgh Steelers,Atlanta Falcons,Pittsburgh Steelers,Atlanta Falcons,18,10,270,0,226,3,Sunday Afternoon
3,2024_01_ARI_BUF,2024,REG,1,2024-09-08,Sunday,01:00PM,ARI,28.0,BUF,34.0,Home,6.0,62.0,0.0,7,7,250.0,-310.0,6.5,-105.0,-115.0,46.0,-112.0,-108.0,0,outdoors,grass,61.0,20.0,Kyler Murray,Josh Allen,Jonathan Gannon,Sean McDermott,Tra Blake,New Era Field,9,REG,Arizona Cardinals,Buffalo Bills,Buffalo Bills,Arizona Cardinals,34,28,352,1,270,1,Sunday Afternoon
4,2024_01_TEN_CHI,2024,REG,1,2024-09-08,Sunday,01:00PM,TEN,17.0,CHI,24.0,Home,7.0,41.0,0.0,7,7,164.0,-198.0,4.0,-108.0,-112.0,43.0,-110.0,-110.0,0,outdoors,grass,67.0,8.0,Will Levis,Caleb Williams,Brian Callahan,Matt Eberflus,Shawn Smith,Soldier Field,9,REG,Tennessee Titans,Chicago Bears,Chicago Bears,Tennessee Titans,24,17,148,1,244,3,Sunday Afternoon
5,2024_01_NE_CIN,2024,REG,1,2024-09-08,Sunday,01:00PM,NWE,16.0,CIN,10.0,Home,-6.0,26.0,0.0,7,7,295.0,-375.0,8.0,-110.0,-110.0,40.5,-112.0,-108.0,0,outdoors,grass,66.0,5.0,Jacoby Brissett,Joe Burrow,Jerod Mayo,Zac Taylor,Land Clark,Paycor Stadium,9,REG,New England Patriots,Cincinnati Bengals,New England Patriots,Cincinnati Bengals,16,10,290,0,224,2,Sunday Afternoon
6,2024_01_HOU_IND,2024,REG,1,2024-09-08,Sunday,01:00PM,HOU,29.0,IND,27.0,Home,-2.0,56.0,0.0,7,7,-155.0,130.0,-3.0,-108.0,-112.0,49.0,-112.0,-108.0,1,closed,artificial,0.0,0.0,C.J. Stroud,Anthony Richardson,DeMeco Ryans,Shane Steichen,John Hussey,Lucas Oil Stadium,9,REG,Houston Texans,Indianapolis Colts,Houston Texans,Indianapolis Colts,29,27,417,0,303,1,Sunday Afternoon
7,2024_01_JAX_MIA,2024,REG,1,2024-09-08,Sunday,01:00PM,JAX,17.0,MIA,20.0,Home,3.0,37.0,0.0,7,7,150.0,-180.0,3.5,-110.0,-110.0,49.5,-112.0,-108.0,0,outdoors,grass,91.0,10.0,Trevor Lawrence,Tua Tagovailoa,Doug Pederson,Mike McDaniel,Craig Wrolstad,Hard Rock Stadium,9,REG,Jacksonville Jaguars,Miami Dolphins,Miami Dolphins,Jacksonville Jaguars,20,17,400,0,267,1,Sunday Afternoon
8,2024_01_CAR_NO,2024,REG,1,2024-09-08,Sunday,01:00PM,CAR,10.0,NOR,47.0,Home,37.0,57.0,0.0,7,7,160.0,-192.0,3.5,-108.0,-112.0,41.5,-112.0,-108.0,1,dome,artificial,0.0,0.0,Bryce Young,Derek Carr,Dave Canales,Dennis Allen,Alan Eck,Mercedes-Benz Superdome,9,REG,Carolina Panthers,New Orleans Saints,New Orleans Saints,Carolina Panthers,47,10,379,1,193,3,Sunday Afternoon
9,2024_01_MIN_NYG,2024,REG,1,2024-09-08,Sunday,01:00PM,MIN,28.0,NYG,6.0,Home,-22.0,34.0,0.0,7,7,-115.0,-105.0,-1.0,-108.0,-112.0,42.5,-108.0,-112.0,0,outdoors,grass,64.0,10.0,Sam Darnold,Daniel Jones,Kevin O'Connell